In [85]:
import sympy as sp
import numpy as np
import math
from IPython.display import display, Math
def sp_skew(v):
   return sp.Matrix([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])


def sp_Transformation_matrix(theta , **kwargs):
    dim = 3
    if type(theta) == int or type(theta) == float:
        theta = theta/180*math.pi
    try:
        if kwargs['angle']=='rad':
            theta = theta/math.pi*180
    except KeyError:
        pass

    try:
        if kwargs['dim']!=0:
            dim = kwargs['dim']
    except KeyError:
        pass

    if dim ==3:
        try:
            if kwargs['axis']=='z':
                return sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],[sp.sin(theta),sp.cos(theta),0],[0,0,1]])
            elif kwargs['axis']=='y':
                return sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],[0,1,0],[-sp.sin(theta),0,sp.cos(theta)]])
            elif kwargs['axis']=='x':
                return sp.Matrix([[1,0,0],[0,sp.cos(theta),-sp.sin(theta)],[0,sp.sin(theta),sp.cos(theta)]])
        except:
            return sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],[sp.sin(theta),sp.cos(theta),0],[0,0,1]])
    elif dim==2:
        return sp.Matrix([[sp.cos(theta),-sp.sin(theta)],[sp.sin(theta),sp.cos(theta)]])
    

t = sp.symbols('t')

from sympy import N, simplify
from sympy.core.numbers import Float

def zero_small(expr, tol=1e-12):
    return expr.replace(
        lambda x: isinstance(x, Float) and abs(x) < tol,
        lambda x: 0
    )
def evaluate(expr, tol=1e-12, digits=2):
    return expr.replace(
        lambda x: isinstance(x, sp.Float),
        lambda x: 0 if abs(x) < tol else round(x, digits)
    )


Kinematics

In [ ]:
q = np.array([0,np.pi/4])
q_dot = np.array([10, 10])
q_ddot = np.array([0,0])

M = 2
l = 3
I = 1/12 * M * l**2

print("parameters")
display(Math(r'q = ' + sp.latex(q) + r'\quad \dot{q} = ' + sp.latex(q_dot) + r'\quad \ddot{q} = ' + sp.latex(q_ddot) + r'\quad M = ' + str(M) + r'\quad l = ' + str(l) + r'\quad I = ' + str(I)))

A1 = sp_Transformation_matrix(q[0], axis='z')
A2 = A1*sp_Transformation_matrix(q[1], axis='y')

display(Math(r'A1 = ' + sp.latex(A1) + r'\quad A2 = ' + sp.latex(A2.evalf(3))))

s1 = sp.Matrix([0 , l , 0])

r1 = sp.Matrix([0, l/2 , 0  ])
r2 = sp.Matrix([0  , 0 , l/2])

w1p = sp.Matrix([0,0,q_dot[0]])
w2p = sp.Matrix([0,q_dot[1],0])

alpha1p = sp.Matrix([0,0,q_ddot[0]])
alpha2p = sp.Matrix([0,q_ddot[1],0])

w1 = w1p
w2 = w1 + A1 * w2p

alpha1 = alpha1p
alpha2 = alpha1 + sp_skew(w1) * A1 * w2p + A1 * alpha2p


a1 = sp_skew(alpha1) * A1 * r1 + sp_skew(w1) * sp_skew(w1) * A1 * r1
display(Math(r'Symbolic \; a_1: \tilde{a}_1 s_1 + \tilde{w_1} \tilde{w_1} s_1 = a_1'))
display(Math(r'Components \; a_1: ' + sp.latex(sp_skew(alpha1) * A1 * r1) + r' + ' + sp.latex(sp_skew(w1) * sp_skew(w1) * A1 * r1)))
a2 = (sp_skew(alpha1) * A1 * s1 + sp_skew(w1) * sp_skew(w1) * A1 * s1 + 
      sp_skew(alpha2) * A2 * r2 + sp_skew(w2) * sp_skew(w2) * A2 * r2)
display(Math(r'Symbolic \; a_2: \tilde{a}_1 s_1 + \tilde{w_1} \tilde{w_1} s_1 + \tilde{a_2} r_2 + \tilde{w_2} \tilde{w_2} r_2 = a_2'))
display(Math(r'Components \; a_2: ' + sp.latex(sp_skew(alpha1) * A1 * s1) + r' + ' + sp.latex(sp_skew(w1) * sp_skew(w1) * A1 * s1) + r' + ' + sp.latex(sp_skew(alpha2) * A2 * r2) + r' + ' + sp.latex(sp_skew(w2) * sp_skew(w2) * A2 * r2)))

print("kinematics")

display(Math(r'w1 = ' + sp.latex(w1) + r'\quad w2 = ' + sp.latex(w2) + r'\quad \alpha1 = ' + sp.latex(alpha1) + r'\quad \alpha2 = ' + sp.latex(alpha2)))
display(Math(r'a1 = ' + sp.latex(a1) + r'\quad a2 = ' + sp.latex(a2.evalf(3)) ))

parameters


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

kinematics


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Dynamics

In [87]:
# Definitions
M1 = sp.Matrix([[M, 0, 0], [0, M, 0], [0, 0, M]])
M2 = sp.Matrix([[M, 0, 0], [0, M, 0], [0, 0, M]])

I1 = A1 * sp.Matrix([[I, 0, 0], [0, 0, 0], [0, 0, I]]) * A1.inv()
I2 = A2 * sp.Matrix([[I, 0, 0], [0, I, 0], [0, 0, 0]]) * A2.inv()

display(Math(r'M1 = ' + sp.latex(M1) +
             r'\quad M2 = ' + sp.latex(M2) +
             r'\quad I1 = ' + sp.latex(evaluate(A1 * I1)) +
             r'\quad I2 = ' + sp.latex(evaluate(I2))))

g = sp.Matrix([0, 0, -9.81])

Force2 = - M2 * a2 + M2 * g


display(Math(r'Symbolic \; R_2: \quad \quad -m \cdot a_2 + m \cdot g = F_2'))
display(Math(r'Componants \; of \; R2: ' +
             sp.latex(evaluate(-M2 * a2)) + r' + ' +
             sp.latex(evaluate(M2 * g)) + r' = ' +
             sp.latex(evaluate(Force2))))

Torque2 = -I2 * alpha2 - sp_skew(w2) * I2 * w2 + sp_skew(A2 * (-r2)) * (-Force2)


display(Math(r'Symbolic \; M_2: \quad \quad \quad I \cdot \alpha_2 +             \omega_2 \times I_2 \cdot \omega_2 +                       (-r_2) \times (-R_2) =                              T_2'))

display(Math(r'Componants \; of \; M_2: ' +
             sp.latex(evaluate(I2 * alpha2)) + r' + ' +
             sp.latex(evaluate(sp_skew(w2) * I2 * w2)) + r' + ' +
             sp.latex(evaluate(sp_skew(A2 * (-r2)) * (-Force2))) + r' = ' +
             sp.latex(evaluate(Torque2))))

display(A2.inv() * (-r2))
Force1 = -M1 * a1 + Force2 + M1 * g

display(Math(r'Symbolic \; R_1: \quad \quad -m \cdot a_1 + F_2 + m \cdot g = F_1'))

display(Math(r'Componants \; of \; R1: ' +
             sp.latex(evaluate(-M1 * a1)) + r' + ' +
             sp.latex(evaluate(Force2)) + r' + ' +
             sp.latex(evaluate(M1 * g)) + r' = ' +
             sp.latex(evaluate(Force1))))

Torque1 = -I1 * alpha1 - sp_skew(w1) * I1 * w1 + sp_skew(A1 * r1) * (Force2) + sp_skew(A1 * (-r1)) * (-Force1) + Torque2

display(Math(r'Symbolic \; M_1: \quad \quad I \cdot \alpha_1 + \omega_1 \times I_1 \cdot \omega_1 - r_1 \times F_2 + r_1 \times F_1 + T_2 = T_1'))

display(Math(r'Componants \; of \; M_1: ' +
             sp.latex(evaluate(I1 * alpha1)) + r' + ' +
             sp.latex(evaluate(sp_skew(w1) * I1 * w1)) + r' - ' +
             sp.latex(evaluate(sp_skew(A1 * r1) * Force2)) + r' - ' +
             sp.latex(evaluate(sp_skew(A1 * (-r1)) * (-Force1))) + r' + ' +
             sp.latex(evaluate(Torque2)) + r' = ' +
             sp.latex(evaluate(Torque1))))

print("Equations of motion (symbolic)")
display(Math(r'F = m \cdot a \quad T = I \cdot \alpha + \omega \times I \cdot \omega + r \times m \cdot a'))

print("forces and torques")
display(Math(
    r'F1 = ' + sp.latex(evaluate(A1.inv()*Force1)) +
    r'\quad M1 = ' + sp.latex(evaluate(A1.inv()*Torque1)) +
    r'\quad F2 = ' + sp.latex(evaluate(A2.inv()*Force2)) +
    r'\quad M2 = ' + sp.latex(evaluate(A2.inv()*Torque2))
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

Equations of motion (symbolic)


<IPython.core.display.Math object>

forces and torques


<IPython.core.display.Math object>

In [88]:
I1 = 1/12 * M * l**2  +  M * (l/2)**2
I2 = 1/12 * M * l**2  +  M * (np.sqrt(l**2+l**2/4))**2
I = I1 + I2
torque = q_ddot[0] * I1 + q_ddot[0]*I2

print("torque")
print(q_ddot[0]*I1)
print(q_ddot[0]*I2)
print(torque)

print(M*g)
print(M*(-9.81)*l/2*np.cos(q[0]))



torque
30.0
120.00000000000001
150.0
Matrix([[0], [0], [-19.6200000000000]])
-20.810152570320096
